# Recurring Events

Expressing recurring events (meetings, alarms, tasks) is a fairly common task, but is complicated by the realities of the calendar system and many of the other things we've discussed in the tutorial so far. The closest thing to a standard way to express recurring events is defined in the iCalendar Spec: [RFC 5545](https://tools.ietf.org/html/rfc5545): "Internet Calendaring and Scheduling Core Object Specification".

![RFC 5545](images/RFC5545-toc.png)

The iCalendar spec is a long document with a nested hierarchy of objects, calendars, events, alarms, TODOs, etc. Buried in that definition in section 3.3.10 is the definition of the *recurrence rule* data type for expressing recurring events.

`dateutil` provides an implementation of this data type in the `rrule` module (partially compliant with RFC5545 and its amendments, originally written to be compliant with RFC 2445). In this section, we will cover how to use the `dateutil.rrule` module.

### RRULE components

The fundamental elements of an `rrule` are:

- `dtstart`: The start point of the recurrence (this is similar to a phase)
- `freq`: The units of the fundamental frequency of the recurrence. It takes the values `YEARLY`, `MONTHLY`, `WEEKLY`, `DAILY`, `HOURLY`, `MINUTELY`, `SECONDLY`
- `interval`: The fundamental frequency of the recurrence, in units of `freq`. If unspecified, this is 1.

To see how these affect the rrule, we'll play around with an `HOURLY` rrule.

In [1]:
from datetime import datetime

from dateutil.rrule import rrule
from dateutil.rrule import MO, TU, WE, TH, FR, SA, SU
from dateutil.rrule import YEARLY, MONTHLY, WEEKLY, DAILY, HOURLY, MINUTELY, SECONDLY

from helper_functions import print_dtlist

In [2]:
hourly = rrule(freq=HOURLY, interval=1, dtstart=datetime(2016, 7, 18, 9), count=3)
print_dtlist(hourly)

2016-07-18 09:00:00
2016-07-18 10:00:00
2016-07-18 11:00:00


In [3]:
dtstart_rr = hourly.replace(dtstart=datetime(2016, 7, 18, 10))
print_dtlist(dtstart_rr)

2016-07-18 10:00:00
2016-07-18 11:00:00
2016-07-18 12:00:00


In [4]:
interval_2 = hourly.replace(interval=2)
print_dtlist(interval_2)

2016-07-18 09:00:00
2016-07-18 11:00:00
2016-07-18 13:00:00


### Limiting RRULEs
An `rrule` starts at `dtstart` (it cannot run backwards), and by default will generate dates indefinitely (in practice, limited by the point at which the `datetime` type cannot represent the dates it would return). There are two, mutually exclusive ways to set limits on the rules:

- `count`: A total number of valid instances to generate
- `until`: The maximum `datetime` value that can be emitted by the `rrule`

### Retrieving subsets
A fairly common operation is to retrieve a subset of the valid recurrences of an `rrule` based on a date. ``rule`` exposes several methods to achieve this:

- `after`: Retrieve the *first* recurrence of the rule *after* the provided datetime
- `before`: Retrieve the *last* recurrence of the rule *before* the provided datetime
- `between`: Retrieve all recurrences of the rule between two datetimes

For example:

In [9]:
rr = rrule(freq=MONTHLY, interval=4, dtstart=datetime(1995, 7, 10))

In [10]:
rr.after(datetime(2015, 2, 1))

datetime.datetime(2015, 3, 10, 0, 0)

In [7]:
rr.before(datetime(1997, 1, 1))

datetime.datetime(1996, 11, 10, 0, 0)

In [8]:
rr.between(datetime(2001, 1, 1), datetime(2002, 1, 1))

[datetime.datetime(2001, 3, 10, 0, 0),
 datetime.datetime(2001, 7, 10, 0, 0),
 datetime.datetime(2001, 11, 10, 0, 0)]

### Bonus Exercise: Play around with some rrules
Before we get into more complicated rules, take a few minutes to practice creating `rrule`s and using them.

Examples:

- Every day at 13:30
- Every other week for 6 weeks (try this with both `count` and `until`)
- What happens when you call `after` with a datetime after the last recurrence? What about `before`?
- How does `between` interact with `count` and `until`?

In [15]:
rr = rrule(freq=DAILY, interval=1, dtstart=datetime(2019, 5, 1, 13, 30), count=12)

In [16]:
print_dtlist(rr)

2019-05-01 13:30:00
2019-05-02 13:30:00
2019-05-03 13:30:00
2019-05-04 13:30:00
2019-05-05 13:30:00
2019-05-06 13:30:00
2019-05-07 13:30:00
2019-05-08 13:30:00
2019-05-09 13:30:00
2019-05-10 13:30:00
2019-05-11 13:30:00
2019-05-12 13:30:00


## `byxxx` rules

So far, we haven't seen any rules that couldn't be implemented (fairly) easily by applying `relativedelta`s to a base date, but we often want to express more common recurrences, like "the third Friday of every month" or "the 1st of each month at 12:30", for these, we need to bring in "by" rules.

`byxxx` rules serve to modify the frequency of the recurrence in some way. The supported rules are `bymonth`, `bymonthday`, `byyearday`, `byweekno`, `byweekday`, `byhour`, `byminute` and `bysecond`, `bysetpos` and `byeaster`.

- When the `byxxx` rule specifies a component *greater than or equal to `freq`*, it is a constraint, and (generally) will *reduce* the frequency of the occurrence. For example:

In [20]:
# Weekend Birthdays
rr = rrule(freq=YEARLY, bymonth=8, bymonthday=8, byweekday=(SA, SU, ),
           dtstart=datetime(1988, 8, 8), count=50)

print_dtlist(rr)

1992-08-08 00:00:00
1993-08-08 00:00:00
1998-08-08 00:00:00
1999-08-08 00:00:00
2004-08-08 00:00:00
2009-08-08 00:00:00
2010-08-08 00:00:00
2015-08-08 00:00:00
2020-08-08 00:00:00
2021-08-08 00:00:00
2026-08-08 00:00:00
2027-08-08 00:00:00
2032-08-08 00:00:00
2037-08-08 00:00:00
2038-08-08 00:00:00
2043-08-08 00:00:00
2048-08-08 00:00:00
2049-08-08 00:00:00
2054-08-08 00:00:00
2055-08-08 00:00:00
2060-08-08 00:00:00
2065-08-08 00:00:00
2066-08-08 00:00:00
2071-08-08 00:00:00
2076-08-08 00:00:00
2077-08-08 00:00:00
2082-08-08 00:00:00
2083-08-08 00:00:00
2088-08-08 00:00:00
2093-08-08 00:00:00
2094-08-08 00:00:00
2099-08-08 00:00:00
2100-08-08 00:00:00
2105-08-08 00:00:00
2106-08-08 00:00:00
2111-08-08 00:00:00
2116-08-08 00:00:00
2117-08-08 00:00:00
2122-08-08 00:00:00
2123-08-08 00:00:00
2128-08-08 00:00:00
2133-08-08 00:00:00
2134-08-08 00:00:00
2139-08-08 00:00:00
2144-08-08 00:00:00
2145-08-08 00:00:00
2150-08-08 00:00:00
2151-08-08 00:00:00
2156-08-08 00:00:00
2161-08-08 00:00:00


In [17]:
# Base frequency is DAILY, but restricted to Tuesdays in November
rr = rrule(freq=DAILY, bymonth=11, byweekday=(TU, ),
           dtstart=datetime(2015, 1, 1, 12), count=5)

print_dtlist(rr)

2015-11-03 12:00:00
2015-11-10 12:00:00
2015-11-17 12:00:00
2015-11-24 12:00:00
2016-11-01 12:00:00


- When the `byxxx` rule is *less than* the frequency, it will generally *increase* the frequency of the recurrence:

In [21]:
# On the 1st, 15th and 30th of every month
rr = rrule(freq=MONTHLY, bymonthday=(1, 15, 30),
           dtstart=datetime(2015, 1, 16, 12, 15), count=4)

print_dtlist(rr)

2015-01-30 12:15:00
2015-02-01 12:15:00
2015-02-15 12:15:00
2015-03-01 12:15:00


### `byweekday`
Similar to weekday constants in the `relativedelta` module, `rrule` also allows specifying rules by weekday. In this case the arguments to the weekday constants are interpreted as the index of all weekdays in one period of the `freq`.

So, for example, if you specify `byweekday=(FR(1), FR(-1))`, with a `MONTHLY` rule, you can get the first and last Friday of the month:

In [22]:
rr = rrule(freq=MONTHLY, byweekday=(FR(1), FR(-1)),
           dtstart=datetime(2021, 1, 1), count=4)

print_dtlist(rr)

2021-01-01 00:00:00
2021-01-29 00:00:00
2021-02-05 00:00:00
2021-02-26 00:00:00


In [26]:
# First Wednesday of the month -- SLCPython meetup
rr = rrule(freq=MONTHLY, byweekday=(WE(1),),
           dtstart=datetime(2014, 2, 1, 19, 30), count=100)
print_dtlist(rr)

2014-02-05 19:30:00
2014-03-05 19:30:00
2014-04-02 19:30:00
2014-05-07 19:30:00
2014-06-04 19:30:00
2014-07-02 19:30:00
2014-08-06 19:30:00
2014-09-03 19:30:00
2014-10-01 19:30:00
2014-11-05 19:30:00
2014-12-03 19:30:00
2015-01-07 19:30:00
2015-02-04 19:30:00
2015-03-04 19:30:00
2015-04-01 19:30:00
2015-05-06 19:30:00
2015-06-03 19:30:00
2015-07-01 19:30:00
2015-08-05 19:30:00
2015-09-02 19:30:00
2015-10-07 19:30:00
2015-11-04 19:30:00
2015-12-02 19:30:00
2016-01-06 19:30:00
2016-02-03 19:30:00
2016-03-02 19:30:00
2016-04-06 19:30:00
2016-05-04 19:30:00
2016-06-01 19:30:00
2016-07-06 19:30:00
2016-08-03 19:30:00
2016-09-07 19:30:00
2016-10-05 19:30:00
2016-11-02 19:30:00
2016-12-07 19:30:00
2017-01-04 19:30:00
2017-02-01 19:30:00
2017-03-01 19:30:00
2017-04-05 19:30:00
2017-05-03 19:30:00
2017-06-07 19:30:00
2017-07-05 19:30:00
2017-08-02 19:30:00
2017-09-06 19:30:00
2017-10-04 19:30:00
2017-11-01 19:30:00
2017-12-06 19:30:00
2018-01-03 19:30:00
2018-02-07 19:30:00
2018-03-07 19:30:00


And if you make it a `YEARLY`, you get the first and last Friday of the *year*:

In [27]:
rr_y = rr.replace(freq=YEARLY)
print_dtlist(rr_y)

2015-01-07 19:30:00
2016-01-06 19:30:00
2017-01-04 19:30:00
2018-01-03 19:30:00
2019-01-02 19:30:00
2020-01-01 19:30:00
2021-01-06 19:30:00
2022-01-05 19:30:00
2023-01-04 19:30:00
2024-01-03 19:30:00
2025-01-01 19:30:00
2026-01-07 19:30:00
2027-01-06 19:30:00
2028-01-05 19:30:00
2029-01-03 19:30:00
2030-01-02 19:30:00
2031-01-01 19:30:00
2032-01-07 19:30:00
2033-01-05 19:30:00
2034-01-04 19:30:00
2035-01-03 19:30:00
2036-01-02 19:30:00
2037-01-07 19:30:00
2038-01-06 19:30:00
2039-01-05 19:30:00
2040-01-04 19:30:00
2041-01-02 19:30:00
2042-01-01 19:30:00
2043-01-07 19:30:00
2044-01-06 19:30:00
2045-01-04 19:30:00
2046-01-03 19:30:00
2047-01-02 19:30:00
2048-01-01 19:30:00
2049-01-06 19:30:00
2050-01-05 19:30:00
2051-01-04 19:30:00
2052-01-03 19:30:00
2053-01-01 19:30:00
2054-01-07 19:30:00
2055-01-06 19:30:00
2056-01-05 19:30:00
2057-01-03 19:30:00
2058-01-02 19:30:00
2059-01-01 19:30:00
2060-01-07 19:30:00
2061-01-05 19:30:00
2062-01-04 19:30:00
2063-01-03 19:30:00
2064-01-02 19:30:00


For any `freq` smaller than a month, the arguments to the weekday are ignored, and you simply get all Fridays:

In [ ]:
rr_w = rr.replace(freq=WEEKLY)
print_dtlist(rr_w)

### Example: Martin Luther King Day

[Martin Luther King, Jr Day](https://en.wikipedia.org/wiki/Martin_Luther_King_Jr._Day) is a US holiday that occurs every year on the third Monday in January. Write a recurrence rule that genrates Martin Luther King Day, starting from its first observance in 1986.

In [ ]:
import rr_answers, rr_tests

In [33]:
### Replace this with your own definition of MLK_DAY
mlk_day = rrule(freq=YEARLY, byweekday=(MO(3),),
           dtstart=datetime(1986, 1, 1))
# mlk_day = rrule(freq=YEARLY, byweekday=(MO(3),), bymonth=1,
#            dtstart=datetime(1986, 1, 1), count=100)
print_dtlist(mlk_day)

1986-01-20 00:00:00
1987-01-19 00:00:00
1988-01-18 00:00:00
1989-01-16 00:00:00
1990-01-15 00:00:00
1991-01-21 00:00:00
1992-01-20 00:00:00
1993-01-18 00:00:00
1994-01-17 00:00:00
1995-01-16 00:00:00
1996-01-15 00:00:00
1997-01-20 00:00:00
1998-01-19 00:00:00
1999-01-18 00:00:00
2000-01-17 00:00:00
2001-01-15 00:00:00
2002-01-21 00:00:00
2003-01-20 00:00:00
2004-01-19 00:00:00
2005-01-17 00:00:00
2006-01-16 00:00:00
2007-01-15 00:00:00
2008-01-21 00:00:00
2009-01-19 00:00:00
2010-01-18 00:00:00
2011-01-17 00:00:00
2012-01-16 00:00:00
2013-01-21 00:00:00
2014-01-20 00:00:00
2015-01-19 00:00:00
2016-01-18 00:00:00
2017-01-16 00:00:00
2018-01-15 00:00:00
2019-01-21 00:00:00
2020-01-20 00:00:00
2021-01-18 00:00:00
2022-01-17 00:00:00
2023-01-16 00:00:00
2024-01-15 00:00:00
2025-01-20 00:00:00
2026-01-19 00:00:00
2027-01-18 00:00:00
2028-01-17 00:00:00
2029-01-15 00:00:00
2030-01-21 00:00:00
2031-01-20 00:00:00
2032-01-19 00:00:00
2033-01-17 00:00:00
2034-01-16 00:00:00
2035-01-15 00:00:00


3824-01-19 00:00:00
3825-01-17 00:00:00
3826-01-16 00:00:00
3827-01-15 00:00:00
3828-01-21 00:00:00
3829-01-19 00:00:00
3830-01-18 00:00:00
3831-01-17 00:00:00
3832-01-16 00:00:00
3833-01-21 00:00:00
3834-01-20 00:00:00
3835-01-19 00:00:00
3836-01-18 00:00:00
3837-01-16 00:00:00
3838-01-15 00:00:00
3839-01-21 00:00:00
3840-01-20 00:00:00
3841-01-18 00:00:00
3842-01-17 00:00:00
3843-01-16 00:00:00
3844-01-15 00:00:00
3845-01-20 00:00:00
3846-01-19 00:00:00
3847-01-18 00:00:00
3848-01-17 00:00:00
3849-01-15 00:00:00
3850-01-21 00:00:00
3851-01-20 00:00:00
3852-01-19 00:00:00
3853-01-17 00:00:00
3854-01-16 00:00:00
3855-01-15 00:00:00
3856-01-21 00:00:00
3857-01-19 00:00:00
3858-01-18 00:00:00
3859-01-17 00:00:00
3860-01-16 00:00:00
3861-01-21 00:00:00
3862-01-20 00:00:00
3863-01-19 00:00:00
3864-01-18 00:00:00
3865-01-16 00:00:00
3866-01-15 00:00:00
3867-01-21 00:00:00
3868-01-20 00:00:00
3869-01-18 00:00:00
3870-01-17 00:00:00
3871-01-16 00:00:00
3872-01-15 00:00:00
3873-01-20 00:00:00


6149-01-20 00:00:00
6150-01-19 00:00:00
6151-01-18 00:00:00
6152-01-17 00:00:00
6153-01-15 00:00:00
6154-01-21 00:00:00
6155-01-20 00:00:00
6156-01-19 00:00:00
6157-01-17 00:00:00
6158-01-16 00:00:00
6159-01-15 00:00:00
6160-01-21 00:00:00
6161-01-19 00:00:00
6162-01-18 00:00:00
6163-01-17 00:00:00
6164-01-16 00:00:00
6165-01-21 00:00:00
6166-01-20 00:00:00
6167-01-19 00:00:00
6168-01-18 00:00:00
6169-01-16 00:00:00
6170-01-15 00:00:00
6171-01-21 00:00:00
6172-01-20 00:00:00
6173-01-18 00:00:00
6174-01-17 00:00:00
6175-01-16 00:00:00
6176-01-15 00:00:00
6177-01-20 00:00:00
6178-01-19 00:00:00
6179-01-18 00:00:00
6180-01-17 00:00:00
6181-01-15 00:00:00
6182-01-21 00:00:00
6183-01-20 00:00:00
6184-01-19 00:00:00
6185-01-17 00:00:00
6186-01-16 00:00:00
6187-01-15 00:00:00
6188-01-21 00:00:00
6189-01-19 00:00:00
6190-01-18 00:00:00
6191-01-17 00:00:00
6192-01-16 00:00:00
6193-01-21 00:00:00
6194-01-20 00:00:00
6195-01-19 00:00:00
6196-01-18 00:00:00
6197-01-16 00:00:00
6198-01-15 00:00:00


7993-01-18 00:00:00
7994-01-17 00:00:00
7995-01-16 00:00:00
7996-01-15 00:00:00
7997-01-20 00:00:00
7998-01-19 00:00:00
7999-01-18 00:00:00
8000-01-17 00:00:00
8001-01-15 00:00:00
8002-01-21 00:00:00
8003-01-20 00:00:00
8004-01-19 00:00:00
8005-01-17 00:00:00
8006-01-16 00:00:00
8007-01-15 00:00:00
8008-01-21 00:00:00
8009-01-19 00:00:00
8010-01-18 00:00:00
8011-01-17 00:00:00
8012-01-16 00:00:00
8013-01-21 00:00:00
8014-01-20 00:00:00
8015-01-19 00:00:00
8016-01-18 00:00:00
8017-01-16 00:00:00
8018-01-15 00:00:00
8019-01-21 00:00:00
8020-01-20 00:00:00
8021-01-18 00:00:00
8022-01-17 00:00:00
8023-01-16 00:00:00
8024-01-15 00:00:00
8025-01-20 00:00:00
8026-01-19 00:00:00
8027-01-18 00:00:00
8028-01-17 00:00:00
8029-01-15 00:00:00
8030-01-21 00:00:00
8031-01-20 00:00:00
8032-01-19 00:00:00
8033-01-17 00:00:00
8034-01-16 00:00:00
8035-01-15 00:00:00
8036-01-21 00:00:00
8037-01-19 00:00:00
8038-01-18 00:00:00
8039-01-17 00:00:00
8040-01-16 00:00:00
8041-01-21 00:00:00
8042-01-20 00:00:00


In [34]:
import rr_tests

### Uncomment to test
rr_tests.test_mlk_day(mlk_day)

Passed!
